In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy.integrate
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.signal as signal
import random
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import utils
import h5py
import time
from sklearn.preprocessing import MinMaxScaler

Using TensorFlow backend.


In [2]:
def fit_func(x, A, B, tau_1, tau_2):
    return A*np.exp(-(x)/tau_1) +B*np.exp(-(x)/tau_2)


class DataObject:

    
    
    def __init__(self, filename):
        self.dt=0.4
        self.file=h5py.File(filename,'r')
        
        self.channel_2_raw_waveforms=self.get_channel_2_raw_waveforms(self.file)
        self.file.close()
        self.channel_2_cut_waveforms, self.half_max_indices, self.min_of_half_max_indices, self.cut_indices=self.get_half_max_indices(self.channel_2_raw_waveforms)
        
        self.lengths=[]
        
        for i in range(len(self.channel_2_cut_waveforms)):
            self.lengths.append(len(self.channel_2_cut_waveforms[i][self.cut_indices[i]:]))
        
        
        self.min_length=min(self.lengths)
        
        
        self.cerenkov_start=4
        self.cerenkov_end=10
        self.channel_2_aligned_waveforms=self.get_channel_2_aligned_waveforms()
        self.channel_2_aligned_waveforms_avg=np.mean(self.channel_2_aligned_waveforms, axis=0)
        
        self.prelim_percentages, self.adjusted_percentages=self.get_percentages()
    
    
    def get_channel_2_raw_waveforms(self, file):
        data = file.get('data')
        data=np.asarray(data)
        digitizer_arrays=[]
        
        for val in data:
            digitizer_arrays.append(val[3][:8192]) #8 channels with 1,024 samples each
        
        digitizer_arrays=np.asarray(digitizer_arrays)
        channel_2_raw_waveforms=[]
        for array in digitizer_arrays:
            pedestal=array[2048]
            channel_2_raw_waveforms.append(-array[2048:3072]+pedestal)
        
        channel_2_raw_waveforms=np.asarray(channel_2_raw_waveforms)
        
        return channel_2_raw_waveforms
    
    def get_half_max_indices(self, channel_2_raw_waveforms):
        channel_2_cut_waveforms=[]
        half_max_indices=[]
        for array in channel_2_raw_waveforms:
            max_val=np.max(array)
            max_val_index=array.argmax()
            
            half_val=max_val/2
            try:
                idx = (np.abs(array[:max_val_index] - half_val)).argmin()
                if idx < 200 and idx>100 and (np.abs(max_val_index-idx)<15) and (max_val>0.05):

                    half_max_indices.append(idx)
                    channel_2_cut_waveforms.append(array)
            except ValueError:
                pass
        channel_2_cut_waveforms=np.asarray(channel_2_cut_waveforms)
        
        
       
        min_of_half_max_indices=min(half_max_indices)
        cut_indices=[]
        
        for i in half_max_indices:
            cut_indices.append(i-min_of_half_max_indices)
            
        return channel_2_cut_waveforms, half_max_indices, min_of_half_max_indices, cut_indices
    
    def get_intermediate_output(self):
        return [self.channel_2_cut_waveforms, self.half_max_indices, self.min_of_half_max_indices, self.min_length]
    
 
    def get_channel_2_aligned_waveforms(self): #, min_of_half_max_indices, min_length):
        self.min_of_half_max_indices=101
        self.min_length=927
        cut_indices=[]
        
        for i in self.half_max_indices:
            cut_indices.append(i-self.min_of_half_max_indices)
        
        channel_2_aligned_waveforms=[]
        for i, array in enumerate(self.channel_2_cut_waveforms):

            aligned_array=array[cut_indices[i]:cut_indices[i]+self.min_length]
            channel_2_aligned_waveforms.append(aligned_array)
        
        channel_2_aligned_waveforms=np.asarray(channel_2_aligned_waveforms)
        return channel_2_aligned_waveforms
    
    def get_percentages(self):
        
        #calculating the extra factor
        xdata=np.arange(0,self.min_length*self.dt,self.dt)
        ydata=self.channel_2_aligned_waveforms_avg

        popt, pcov = sp.optimize.curve_fit(fit_func, xdata[self.min_of_half_max_indices+self.cerenkov_end:], ydata[self.min_of_half_max_indices+self.cerenkov_end:] , p0=(0.26, 0.74,55,145))
        yvals=fit_func(xdata[:], *popt)

        b=sp.integrate.trapz(yvals[self.min_of_half_max_indices+self.cerenkov_end:], xdata[self.min_of_half_max_indices+self.cerenkov_end:])
        c=sp.integrate.trapz(yvals[self.min_of_half_max_indices-self.cerenkov_start:], xdata[self.min_of_half_max_indices-self.cerenkov_start:])
        self.adjustment_factor=b/c
        
        prelim_percentages=[]
        adjusted_percentages=[]
        for array in self.channel_2_aligned_waveforms:
    
            cerenkov=np.trapz(array[self.min_of_half_max_indices-self.cerenkov_start:self.min_of_half_max_indices+self.cerenkov_end], dx=self.dt)
            scint=np.trapz(array[self.min_of_half_max_indices+self.cerenkov_end:], dx=self.dt)
            adjusted_cerenkov=cerenkov-(scint*((1/self.adjustment_factor)-1))
            adjusted_scint=scint/self.adjustment_factor
            #total=adjusted_cerenkov+adjusted_scint
            prelim_percentages.append([cerenkov, scint])
            adjusted_percentages.append([adjusted_cerenkov, adjusted_scint])
        
        prelim_percentages=np.asarray(prelim_percentages)
        adjusted_percentages=np.asarray(adjusted_percentages)
        return prelim_percentages, adjusted_percentages
    
    def final_output(self):
        return [self.channel_2_aligned_waveforms, self.adjusted_percentages]
        

In [3]:
def model_creator():
    model = Sequential()
    #model.add(Dense(50,  input_shape=(300,), kernel_initializer='normal', activation='relu'))
    model.add(Dense(10, input_shape=(927,), kernel_initializer='ones', activation='relu'))
    model.add(Dense(2, kernel_initializer='ones', activation='relu'))
# Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def model_trainer(model_name, pre_X, Y):
    model=model_creator()
        
    

    Y=np.asarray(Y)
    
    X=np.asarray(pre_X)
    
    norm_factor=np.amax(X)
    X_scaled=X/norm_factor
    Xtrain, Xval, Ytrain, Yval = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)
    checkpointer = ModelCheckpoint(filepath=model_name+"weights.hdf5",verbose=3, save_best_only=True)
    earlystop= EarlyStopping(monitor='val_loss', min_delta=0, patience=20 ,verbose=1, mode='auto')
    history=model.fit(Xtrain,Ytrain,epochs=200,verbose=1
                             ,validation_data=(Xval,Yval)
                             ,shuffle=True, batch_size=16
                             ,callbacks=[earlystop,checkpointer])
    model.save(model_name)
    return model, history



def model_verifier(model_name, runs):
    model=load_model(model_name)
    output_dict={}
    for run in runs:
        prediction=model.predict(data_dict[run][1])
        percent_difference=(2*(prediction-data_dict[run][2])/(prediction+data_dict[run][2]))
        percent_difference_avg=np.mean(percent_difference, axis=0)
        output_dict[run]=percent_difference_avg
    
    return output_dict

In [17]:
runs=['6292', '6337']#['6292', '6311', '6314', '6330', '6337']

X=None
Y=None

data_dict={}

for run in runs:
    
    d=DataObject('D:\\Murali Backup\\Documents\\15-Cornell-3rd-year\\2nd-Semester\\Pulse_Generation_and_Neural_Net\\Workspace\\Dual_Readout_Calorimetry\\Real_Data\\beamline_data\\Real_Data\\' + run + '_skimTree.hdf5')
    output_list=d.final_output()
    Xtrain, Xval, Ytrain, Yval = train_test_split(output_list[0], output_list[1], test_size=0.2, random_state=42)
    data_dict[run]=[d, Xval, Yval]
    #print(len(Xtrain))
    #print(len(Xval))
    
    
    if X is None and Y is None:
        X=Xtrain
        Y=Ytrain
        #print('h')
        
    else:
        X=np.concatenate((X, Xtrain))
        Y=np.concatenate((Y, Ytrain))
        #print('p')

In [18]:
model, history=model_trainer('v3_secondtry_6292_6337.hdf5',X,Y)





Train on 51401 samples, validate on 12851 samples
Epoch 1/200
51401/51401 [==============================] - 6s 110us/step - loss: 2667.6348 - val_loss: 16.3433

Epoch 00001: val_loss improved from inf to 16.34326, saving model to v3_secondtry_6292_6337.hdf5weights.hdf5
Epoch 2/200
51401/51401 [==============================] - 6s 121us/step - loss: 4.3148 - val_loss: 0.9722

Epoch 00002: val_loss improved from 16.34326 to 0.97215, saving model to v3_secondtry_6292_6337.hdf5weights.hdf5
Epoch 3/200
51401/51401 [==============================] - 5s 100us/step - loss: 0.6464 - val_loss: 0.4176

Epoch 00003: val_loss improved from 0.97215 to 0.41760, saving model to v3_secondtry_6292_6337.hdf5weights.hdf5
Epoch 4/200
51401/51401 [==============================] - 5s 106us/step - loss: 0.2153 - val_loss: 0.0541

Epoch 00004: val_loss improved from 0.41760 to 0.05409, saving model to v3_secondtry_6292_6337.hdf5weights.hdf5
Epoch 5/200
51401/51401 [==============================] - 5s 95us/s

51401/51401 [==============================] - 4s 82us/step - loss: 0.0258 - val_loss: 0.0085

Epoch 00044: val_loss did not improve from 0.00842
Epoch 45/200
51401/51401 [==============================] - 4s 86us/step - loss: 0.0258 - val_loss: 0.0084

Epoch 00045: val_loss did not improve from 0.00842
Epoch 46/200
51401/51401 [==============================] - 5s 92us/step - loss: 0.0258 - val_loss: 0.0084

Epoch 00046: val_loss did not improve from 0.00842
Epoch 00046: early stopping


In [15]:
output=model_verifier('v3_secondtry_6292_6337.hdf5', runs)

In [16]:
print(output)

{'6330': array([-2.       , -1.1336053]), '6337': array([-2.        , -1.13237874])}


In [ ]:
model=load_model('v3_firsttry.hdf5')

test=[data_dict['6292'][0].channel_2_aligned_waveforms[100]]
test=np.asarray(test)
print(model.predict(test))
print(data_dict['6292'][0].adjusted_percentages[100])
print(data_dict['6292'][0].prelim_percentages[100])
print(1/data_dict['6292'][0].adjustment_factor)

In [ ]:


d1=DataObject('6292_skimTree.hdf5')

In [ ]:
d2=DataObject('D:\\Murali Backup\\Documents\\15-Cornell-3rd-year\\2nd-Semester\\Pulse_Generation_and_Neural_Net\\Workspace\\Dual_Readout_Calorimetry\\Real_Data\\beamline_data\\Real_Data\\6337_skimTree.hdf5')

In [ ]:
a=d1.final_output()
b=d2.final_output()

In [ ]:
a[0].shape

In [ ]:
print(len(a[0])==len(a[1]))

In [ ]:
print(len(b[0])==len(b[1]))

In [ ]:
print(len(a[0][0]))

In [ ]:
print(len(b[0][0]))

In [ ]:
print(a[0])

In [ ]:
X=np.concatenate((a[0],b[0]))
Y=np.concatenate((a[1],b[1]))

In [ ]:
model, history=model_trainer('v3_firsttry',X,Y)

In [ ]:
test=[X[5000]]
test=np.asarray(test)

p=model.predict(test)

In [ ]:
print(2*(Y[5000]-p)/(Y[5000]+p))

In [ ]:
print(p)

In [ ]:
print(Y[5000])

In [ ]:
file=h5py.File('D:\\Murali Backup\\Documents\\15-Cornell-3rd-year\\2nd-Semester\\Pulse_Generation_and_Neural_Net\\Workspace\\Dual_Readout_Calorimetry\\Real_Data\\beamline_data\\Real_Data\\6337_skimTree.hdf5','r')
file.close()